<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Реализуем триграмную модель

In [0]:
from google.colab import files
uploaded = files.upload()

Saving anna_karenina.txt to anna_karenina.txt
Saving besy_dostoevsky.txt to besy_dostoevsky.txt


In [0]:
dostoevsky = open('besy_dostoevsky.txt').read()
tolstoy = open('anna_karenina.txt').read()

In [247]:
print("Длина Бесов Достоевского -", len(dostoevsky))
print("Длина Анны Карениной Толстого - ", len(tolstoy))

Длина Бесов Достоевского - 1293557
Длина Анны Карениной Толстого -  1710408


In [0]:
from string import punctuation
import numpy as np

def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
norm_dostoevsky = normalize(dostoevsky)
norm_tolstoy = normalize(tolstoy)

In [250]:
print("Длина Бесов Достоевского в токенах -", len(norm_dostoevsky))
print("Длина Анны Карениной Толстого в токенах - ", len(norm_tolstoy))

Длина Бесов Достоевского в токенах - 208453
Длина Анны Карениной Толстого в токенах -  281201


In [0]:
from collections import Counter
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [252]:
import nltk
nltk.download('punkt')
sentences_dostoevsky = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_tolstoy = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()

dut = sentences_dostoevsky[:1000]
for sentence in dut:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence))

unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()

cut = sentences_tolstoy[:1000]
for sentence in cut:
    unigrams_tolstoy.update(sentence)
    bigrams_tolstoy.update(ngrammer(sentence))

In [0]:
from collections import Counter
unigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in dut:
    unigrams_dostoevsky.update(sentence)
    trigrams_dostoevsky.update(ngrammer(sentence, n=3))
    
unigrams_tolstoy = Counter()
trigrams_tolstoy = Counter()

for sentence in cut:
    unigrams_tolstoy.update(sentence)
    trigrams_tolstoy.update(ngrammer(sentence, n=3))

In [255]:
print(len(unigrams_dostoevsky))

6115


In [256]:
print(trigrams_tolstoy.most_common(10))

[('<start> <start> –', 393), ('<start> <start> он', 41), ('<start> <start> и', 36), ('– сказал он', 32), ('<start> <start> степан', 30), ('<start> степан аркадьич', 30), ('<start> <start> но', 28), ('<start> – ну', 28), ('<start> <start> она', 27), ('<start> <start> левин', 26)]


In [0]:
matrix_tolstoy = np.zeros((len(unigrams_tolstoy), 
                   len(unigrams_tolstoy)))

id2word_unigrams = list(unigrams_tolstoy)
word2id_unigrams = {word:i for i, word in enumerate(id2word_unigrams)}
id2word_bigrams = list(bigrams_tolstoy)
word2id_bigrams = {coll:d for d, coll in enumerate(id2word_bigrams)}

for ngram in bigrams_tolstoy:
    word1, word2 = ngram.split()
    matrix_tolstoy[word2id_unigrams[word1]][word2id_unigrams[word2]] =  (bigrams_tolstoy[ngram]/
                                                                     unigrams_tolstoy[word1])

In [0]:
matrix_tolstoy = np.zeros((len(bigrams_tolstoy), 
                   len(unigrams_tolstoy)))

for ngram in trigrams_tolstoy:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_tolstoy[word2id_bigrams[bigram]][word2id_unigrams[word3]] =  (trigrams_tolstoy[ngram]/
                                                                     bigrams_tolstoy[bigram])

In [0]:
def generate(matrix, id2word, word2id, n=100, start=['<start>','<start>']):
    text = []
    current_bigram = start
    current_idx = word2id[' '.join(current_bigram)]
    
    for i in range(n):  
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            current_bigram = start
            current_idx = word2id[' '.join(current_bigram)]
        else:
          current_bigram = [current_bigram[1], id2word[chosen]]
          current_idx = word2id[' '.join(current_bigram)]


    
    return ' '.join(text)

In [260]:
print(generate(matrix_tolstoy, id2word_unigrams, word2id_bigrams).replace('<end>', '\n'))

знакомый встретился и окликнул его но левин был его товарищем и другом первой молодости что чувство это не любовь 
 – нет уж не может же это все стесняет – сказал сергей иваныч покачивая головой 
 я огорчил ее 
 авось бог даст 
 но только дикость моя не в том что ты 
 это от вас зависит – сказал он с значительным видом склонив голову набок уставился на барина 
 во время своего студенчества он чуть было не строго глаза смотрели так же как и почему он спит не в силах удерживать улыбки говорил левин 
 – эге 



In [0]:
matrix_dostoevsky = np.zeros((len(unigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))

dosid2word_unigrams = list(unigrams_dostoevsky)
dosword2id_unigrams = {word:i for i, word in enumerate(dosid2word_unigrams)}
dosid2word_bigrams = list(bigrams_dostoevsky)
dosword2id_bigrams = {coll:d for d, coll in enumerate(dosid2word_bigrams)}

for mgram in bigrams_dostoevsky:
    words1, words2 = mgram.split()
    matrix_dostoevsky[dosword2id_unigrams[words1]][dosword2id_unigrams[words2]] =  (bigrams_dostoevsky[mgram]/
                                                                     unigrams_dostoevsky[words1])

In [0]:
matrix_dostoevsky = np.zeros((len(bigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))

for mgram in trigrams_dostoevsky:
    words1, words2, words3 = mgram.split()
    dosbigram = '{} {}'.format(words1, words2)
    matrix_dostoevsky[dosword2id_bigrams[dosbigram]][dosword2id_unigrams[words3]] =  (trigrams_dostoevsky[mgram]/
                                                                     bigrams_dostoevsky[dosbigram])

In [263]:
print(generate(matrix_dostoevsky, dosid2word_unigrams, dosword2id_bigrams).replace('<end>', '\n'))

прибавляли сверх того по другим отчасти уже деликатным причинам 
 шатову очень хотелось ему тогда за значительный карточный проигрыш внезапно обнаружившийся 
 тогда еще не успели ничего понять да им и не думая о ней надо бы поговорить особенно что я развратитель молодежи и рассадник губернского атеизма… он тотчас же по возвращении вознаградить тем более что постоянно проигрывал 
 что может быть и правы вы ведь читали сейчас письмо дроздовой что могло быть удовлетворено 
 она конечно понимала что ей нельзя водиться с этими людьми но все-таки принимала их с жадностию прочла это первое письмо и подчеркнув карандашом восклицание «где


Тексты, сгенерированные при помощи триграммов, больше похожи на человеческую речь, чем тексты, созданные биграммами, так как выбор следующего слова происходит исходя из большего контекста.

2. Gensim.models.Phrases

In [208]:
from nltk.tokenize import sent_tokenize
 
text="""В конце 1811 г., в эпоху нам достопамятную, жил в своем поместье Ненарадове добрый Гаврила Гаврилович Р**. Он славился во всей округе гостеприимством и радушием; соседи поминутно ездили к нему поесть, попить, поиграть по пяти копеек в бостон с его женою, а некоторые для того, чтоб поглядеть на дочку их, Марью Гавриловну, стройную, бледную и семнадцатилетнюю девицу. Она считалась богатой невестою, и многие прочили ее за себя или за сыновей. Марья Гавриловна была воспитана на французских романах, и следственно была влюблена. Предмет, избранный ею, был бедный армейский прапорщик, находившийся в отпуску в своей деревне."""
sentences = sent_tokenize(text)


for sents in sentences:
    print(sents)

В конце 1811 г., в эпоху нам достопамятную, жил в своем поместье Ненарадове добрый Гаврила Гаврилович Р**.
Он славился во всей округе гостеприимством и радушием; соседи поминутно ездили к нему поесть, попить, поиграть по пяти копеек в бостон с его женою, а некоторые для того, чтоб поглядеть на дочку их, Марью Гавриловну, стройную, бледную и семнадцатилетнюю девицу.
Она считалась богатой невестою, и многие прочили ее за себя или за сыновей.
Марья Гавриловна была воспитана на французских романах, и следственно была влюблена.
Предмет, избранный ею, был бедный армейский прапорщик, находившийся в отпуску в своей деревне.


In [244]:
phrases = gensim.models.Phrases(texts, min_count=1, scoring = 'npmi', threshold=0)
p_2 = gensim.models.Phrases(phrases[voc], scoring='npmi', threshold=-1)
p_3 = gensim.models.Phrases(p_[phrases[voc]], scoring='npmi', threshold=-1)
p_4 = gensim.models.Phrases(p3[p2[phrases[voc]]], scoring='npmi', threshold=-1)
print(list(p_4(p_3(p_2[voc]))))

NameError: ignored